In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import json

In [2]:
# purpose: open and read txt file function
def open_read(txt_file):
    # reading the txt data by using JSON
    with open(txt_file, encoding='utf-8') as file:
        json_data = json.load(file)
        # converting json dataset from dictionary to dataframe
        df = pd.DataFrame(json_data)
    return df

In [3]:
traveltime_df = open_read('traveltime.txt')

In [4]:
# create dictionarys - {key=RouteID : value=TravelTimes}
route_traveltimes_dict = {}
for row_index, route_id in enumerate(traveltime_df['RouteID']):
    route_traveltimes_dict[route_id] = pd.json_normalize(traveltime_df['TravelTimes'][row_index])

In [5]:
# purpose: revise linecolor label and fill the whole name
def extract_color(color_ID):
    if color_ID[0:2] == 'BR':
        return 'brown'
    elif color_ID[0:1] == 'R':
        return 'red'
    elif color_ID[0:1] == 'Y':
        return 'yellow'
    elif color_ID[0:1] == 'B' or color_ID[0:2] == 'BL':
        return 'blue'
    elif color_ID[0:1] == 'O':
        return 'orange'
    elif color_ID[0:1] == 'G':
        return 'green'

In [6]:
#  purpose: create timetables with calculating all timevalues
def get_route_timetable(traveltimes_df):
    # add a new column to calculate commute time in each station
    traveltimes_df['TakeTime'] = traveltimes_df['RunTime'] + traveltimes_df['StopTime']
    # add a new column to calculate commute time between two stations
    traveltimes_df['AccTime'] = traveltimes_df['TakeTime'].cumsum()
    # add a new column to classify stations by their linescolors
    traveltimes_df['Line_Color'] = traveltimes_df['FromStationID'].apply(extract_color)
    
    times_df = traveltimes_df.rename(columns={'FromStationName.Zh_tw':'FromStationName', 'ToStationName.Zh_tw':'ToStationName'})
    return times_df

# purpose: create a pivot table - the index and column frame by all station names
def get_route_stationtable(traveltimes_df):
    # if totalstation=n, FromStationName has n-1 station names(coz end station name in ToStationName)
    indexes = np.append(traveltimes_df['FromStationName.Zh_tw'].values, traveltimes_df['ToStationName.Zh_tw'].iloc[-1])
    stationframe_df = pd.DataFrame(index=indexes, columns=indexes)
    return stationframe_df

# purpose: fill all timevalues in the station framtable and search total commute time between two stations
def fill_timevalue(traveltimes_df, stationframe_df):
    
    stop_time = np.array(traveltimes_df['StopTime'])
    # StopTime in the last station is 0
    stop_time = np.append(stop_time, 0)
    take_time = np.array(traveltimes_df['TakeTime'])

    for row_index in range(len(stationframe_df)):
        for column_index in range(len(stationframe_df)):
            if row_index == column_index:
                stationframe_df.iloc[row_index][column_index] = stop_time[row_index]
            elif row_index < column_index:                
                stationframe_df.iloc[row_index][column_index] = take_time[row_index:column_index].sum()
            elif row_index > column_index:
                stationframe_df.iloc[row_index][column_index] = take_time[column_index:row_index].sum()
            else:
                continue
                
    schedule_df = stationframe_df
    return schedule_df

In [7]:
# create dictionarys - {key:routecolor: {key=stationname0: {key=stationnames: value=commutetimes between other stations}}
commutetimes_dict = {}    
for row_index, route_id in enumerate(traveltime_df['RouteID']):
    commutetimes_dict[route_id] = fill_timevalue(get_route_timetable(route_traveltimes_dict[route_id]), get_route_stationtable(route_traveltimes_dict[route_id])).to_dict()

print(commutetimes_dict)

{'BL-1': {'南港展覽館': {'南港展覽館': 0, '南港': 112, '昆陽': 245, '後山埤': 377, '永春': 482, '市政府': 594, '國父紀念館': 699, '忠孝敦化': 798, '忠孝復興': 891, '忠孝新生': 1021, '善導寺': 1142, '台北車站': 1233, '西門': 1395, '龍山寺': 1540, '江子翠': 1761, '新埔': 1866, '板橋': 2001, '府中': 2096, '亞東醫院': 2226, '海山': 2394, '土城': 2526, '永寧': 2647, '頂埔': 2855}, '南港': {'南港展覽館': 112, '南港': 24, '昆陽': 133, '後山埤': 265, '永春': 370, '市政府': 482, '國父紀念館': 587, '忠孝敦化': 686, '忠孝復興': 779, '忠孝新生': 909, '善導寺': 1030, '台北車站': 1121, '西門': 1283, '龍山寺': 1428, '江子翠': 1649, '新埔': 1754, '板橋': 1889, '府中': 1984, '亞東醫院': 2114, '海山': 2282, '土城': 2414, '永寧': 2535, '頂埔': 2743}, '昆陽': {'南港展覽館': 245, '南港': 133, '昆陽': 25, '後山埤': 132, '永春': 237, '市政府': 349, '國父紀念館': 454, '忠孝敦化': 553, '忠孝復興': 646, '忠孝新生': 776, '善導寺': 897, '台北車站': 988, '西門': 1150, '龍山寺': 1295, '江子翠': 1516, '新埔': 1621, '板橋': 1756, '府中': 1851, '亞東醫院': 1981, '海山': 2149, '土城': 2281, '永寧': 2402, '頂埔': 2610}, '後山埤': {'南港展覽館': 377, '南港': 265, '昆陽': 132, '後山埤': 29, '永春': 105, '市政府': 217, '國父紀念館': 322, '忠孝敦化': 421, '忠

In [8]:
# open and read the transfer stations file
linetransfer_df = open_read('linetransfer.txt')
linetransfer_df

,FromLineNo,FromLineID,FromLineName,FromStationID,FromStationName,ToLineNo,ToLineID,ToLineName,ToStationID,ToStationName,IsOnSiteTransfer,TransferTime,TransferDescription,SrcUpdateTime,UpdateTime,VersionID
0,BL,BL,"{'Zh_tw': '板南線', 'En': 'Bannan Line'}",BL23,"{'Zh_tw': '南港展覽館', 'En': 'Taipei Nangang Exhib...",BR,BR,"{'Zh_tw': '文湖線', 'En': 'Wenhu Line'}",BR24,"{'Zh_tw': '南港展覽館', 'En': 'Taipei Nangang Exhib...",1,5,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
1,R,R,"{'Zh_tw': '淡水信義線', 'En': 'Tamsui-Xinyi Line'}",R07,"{'Zh_tw': '東門', 'En': 'Dongmen'}",O,O,"{'Zh_tw': '中和新蘆線', 'En': 'Zhonghe-Xinlu Line'}",O06,"{'Zh_tw': '東門', 'En': 'Dongmen'}",1,2,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
2,R,R,"{'Zh_tw': '淡水信義線', 'En': 'Tamsui-Xinyi Line'}",R11,"{'Zh_tw': '中山', 'En': 'Zhongshan'}",G,G,"{'Zh_tw': '松山新店線', 'En': 'Songshan-Xindian Line'}",G14,"{'Zh_tw': '中山', 'En': 'Zhongshan'}",1,3,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
3,G,G,"{'Zh_tw': '松山新店線', 'En': 'Songshan-Xindian Line'}",G16,"{'Zh_tw': '南京復興', 'En': 'Nanjing Fuxing'}",BR,BR,"{'Zh_tw': '文湖線', 'En': 'Wenhu Line'}",BR11,"{'Zh_tw': '南京復興', 'En': 'Nanjing Fuxing'}",1,5,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
4,G,G,"{'Zh_tw': '松山新店線', 'En': 'Songshan-Xindian Line'}",G15,"{'Zh_tw': '松江南京', 'En': 'Songjiang Nanjing'}",O,O,"{'Zh_tw': '中和新蘆線', 'En': 'Zhonghe-Xinlu Line'}",O08,"{'Zh_tw': '松江南京', 'En': 'Songjiang Nanjing'}",1,2,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
5,O,O,"{'Zh_tw': '中和新蘆線', 'En': 'Zhonghe-Xinlu Line'}",O05,"{'Zh_tw': '古亭', 'En': 'Guting'}",G,G,"{'Zh_tw': '松山新店線', 'En': 'Songshan-Xindian Line'}",G09,"{'Zh_tw': '古亭', 'En': 'Guting'}",1,2,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
6,R,R,"{'Zh_tw': '淡水信義線', 'En': 'Tamsui-Xinyi Line'}",R05,"{'Zh_tw': '大安', 'En': 'Daan'}",BR,BR,"{'Zh_tw': '文湖線', 'En': 'Wenhu Line'}",BR09,"{'Zh_tw': '大安', 'En': 'Daan'}",1,5,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
7,G,G,"{'Zh_tw': '松山新店線', 'En': 'Songshan-Xindian Line'}",G12,"{'Zh_tw': '西門', 'En': 'Ximen'}",BL,BL,"{'Zh_tw': '板南線', 'En': 'Bannan Line'}",BL11,"{'Zh_tw': '西門', 'En': 'Ximen'}",1,2,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
8,BR,BR,"{'Zh_tw': '文湖線', 'En': 'Wenhu Line'}",BR10,"{'Zh_tw': '忠孝復興', 'En': 'Zhongxiao Fuxing'}",BL,BL,"{'Zh_tw': '板南線', 'En': 'Bannan Line'}",BL15,"{'Zh_tw': '忠孝復興', 'En': 'Zhongxiao Fuxing'}",1,5,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
9,BR,BR,"{'Zh_tw': '文湖線', 'En': 'Wenhu Line'}",BR24,"{'Zh_tw': '南港展覽館', 'En': 'Taipei Nangang Exhib...",BL,BL,"{'Zh_tw': '板南線', 'En': 'Bannan Line'}",BL23,"{'Zh_tw': '南港展覽館', 'En': 'Taipei Nangang Exhib...",1,5,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2


In [9]:
def json_normalize(df):
    target_cols = ['FromLineName','FromStationName','ToLineName', 'ToStationName']
    for col in target_cols:
        df[col] = pd.json_normalize(df[col])
    return df

json_normalize(linetransfer_df).head(3)

,FromLineNo,FromLineID,FromLineName,FromStationID,FromStationName,ToLineNo,ToLineID,ToLineName,ToStationID,ToStationName,IsOnSiteTransfer,TransferTime,TransferDescription,SrcUpdateTime,UpdateTime,VersionID
0,BL,BL,板南線,BL23,南港展覽館,BR,BR,文湖線,BR24,南港展覽館,1,5,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
1,R,R,淡水信義線,R07,東門,O,O,中和新蘆線,O06,東門,1,2,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
2,R,R,淡水信義線,R11,中山,G,G,松山新店線,G14,中山,1,3,,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2


In [10]:
linetransfer_df['From_Line_Color'] = linetransfer_df['FromStationID'].apply(extract_color)
linetransfer_df['To_Line_Color'] = linetransfer_df['ToStationID'].apply(extract_color)

# remove two stations which are out of station
out_station_transfer = linetransfer_df[(linetransfer_df['FromStationName']=='新埔') | (linetransfer_df['FromStationName']=='新埔民生') | 
                                       (linetransfer_df['ToStationName']=='新埔') | (linetransfer_df['ToStationName']=='新埔民生') ]
linetransfer_df.drop(out_station_transfer.index, axis=0, inplace=True)

transfer_stations_df = linetransfer_df[['From_Line_Color', 'FromStationName', 'To_Line_Color', 'ToStationName']].sort_values(['From_Line_Color']).reset_index(drop=True)
transfer_stations_df

,From_Line_Color,FromStationName,To_Line_Color,ToStationName
0,blue,南港展覽館,brown,南港展覽館
1,blue,忠孝復興,brown,忠孝復興
2,blue,西門,green,西門
3,blue,忠孝新生,orange,忠孝新生
4,blue,板橋,yellow,板橋
5,blue,台北車站,red,台北車站
6,brown,忠孝復興,blue,忠孝復興
7,brown,南港展覽館,blue,南港展覽館
8,brown,南京復興,green,南京復興
9,brown,大安,red,大安


In [11]:
# purpose: get the line colors of thestation
def get_line_color(station_name):
    # pick up station_df in the MRT station ipynotebook
    station_df = pd.read_pickle("./station.pkl")
    color = station_df[station_df['StationName_Zh_tw']==station_name]['Line_Color']
    return color

# purpose: help to search commute time from one station to another station
def choose_searching_dict(color):
    if color == 'yellow':
        return commutetimes_dict['Y-1']
    elif color == 'brown':
        return commutetimes_dict['BR-1']
    elif color == 'blue':
        return commutetimes_dict['BL-1'] or commutetimes_dict['BL-2']
    elif color == 'red':
        return commutetimes_dict['R-1'] or commutetimes_dict['R-2'] or commutetimes_dict['R-3']
    elif color == 'green':
        return commutetimes_dict['G-1'] or commutetimes_dict['G-2'] or commutetimes_dict['G-3']
    else:
        return commutetimes_dict['O-1'] or commutetimes_dict['O-2']
    
# purpose: get the intersections to show all probable ways we can arrive the end_station
def find_intersections(start_station, end_station):
    
    # get the line colors of start_station and end_station
    start_line_color = get_line_color(start_station).values[0]
    end_line_color = get_line_color(end_station).values[0]
    
    with_start_color= transfer_stations_df[transfer_stations_df['From_Line_Color'] == start_line_color]
    with_end_color= transfer_stations_df[transfer_stations_df['To_Line_Color'] == end_line_color]
    start_intersect_end= pd.merge(with_start_color, with_end_color, left_on='To_Line_Color', right_on='From_Line_Color', suffixes=('_x', '_y'))
    
    return start_intersect_end[['From_Line_Color_x', 'To_Line_Color_x', 'ToStationName_x', 'ToStationName_y', 'To_Line_Color_y']]

In [12]:
def get_total_taketime(start_station, end_station):
    
    # get the line colors of start_station and end_station
    start_line_color = get_line_color(start_station).values[0]
    end_line_color = get_line_color(end_station).values[0]
    
    transfer_table = find_intersections(start_station, end_station)
    
    time_array = []
    for i in range(len(transfer_table)):
        time_sum = 0
        # start_station -> intersect_from_station    
        time_sum = choose_searching_dict(start_line_color)[start_station][transfer_table['ToStationName_x'][i]]
        # intersect_from_station -> intersect_to_station
        time_sum += choose_searching_dict(transfer_table['To_Line_Color_x'][i])[transfer_table['ToStationName_x'][i]][transfer_table['ToStationName_y'][i]]
        # intersect_to_station -> end_station
        time_sum += choose_searching_dict(end_line_color)[transfer_table['ToStationName_y'][i]][end_station]
        
        time_array.append(time_sum)
    
    transfer_table["total_time"] = np.array(time_array)
    
    return transfer_table

In [13]:
get_total_taketime('淡水', '中山國小')

,From_Line_Color_x,To_Line_Color_x,ToStationName_x,ToStationName_y,To_Line_Color_y,total_time
0,red,green,中正紀念堂,松江南京,orange,3307
1,red,green,中正紀念堂,古亭,orange,3353
2,red,green,中山,松江南京,orange,2503
3,red,green,中山,古亭,orange,3519
4,red,blue,台北車站,忠孝新生,orange,2802
5,red,red,北投,民權西路,orange,2049
6,red,red,北投,東門,orange,3192
7,red,orange,民權西路,大橋頭,orange,2245
8,red,orange,東門,大橋頭,orange,3598
